# Aerodynamic Splatting

In [ ]:
import numpy as np
from sympy import symbols, diff, Quaternion, exp, Matrix, lambdify, sin, cos, simplify
from sympy.physics.mechanics import *
from scipy.spatial.transform import Rotation
from IPython.display import display
import matplotlib.pyplot as plt

## Create Symbolic Functions

### Properties of the splat

In [ ]:
x, y, z, u, v, w, sx, sy, sz, qw, qx, qy, qz, θ = symbols("x, y, z, u, v, w, s_x, s_y, s_z, q_w, q_x, q_y, q_z, θ")
position = np.array([x, y, z])
velocity = np.array([u, v, w])
scale = np.array([sx, sy, sz])
orientation_2d = θ
orientation_3d = np.array([qw, qx, qy, qz])
splat_properties_2d = [x, y, u, v, sx, sy, θ]
splat_properties_2d_example = [0, 0] + [1, 0] + [1, 1] + [0] + [2, 0]
splat_properties_3d = [x, y, z, u, v, w, sx, sy, sz, qw, qx, qy, qz]
splat_properties_3d_example = [0, 0, 0] + [1, 0, 0] + [1, 1, 1] + [1, 0, 0, 0] + [2, 0, 0] 

### Position for evaluation of the splat equations

In [ ]:
xp, yp, zp = symbols("x_p, y_p, z_p")
eval_coords_2d = [xp, yp]
eval_coords_3d = [xp, yp, zp]
eval_at_position = np.array(eval_coords_3d)

## Equation for quaternion
We provide two options for obtaining the quaternon.
In the case where a `numpy` array with four quaternion components are provided, we will obtain using the standard syntax `Quaternion(qw, qx, qy, qz)`.
In the case where a single component is provided, we will assume that this is a planar model with an angle evaluated about the $z$-axis, and obtain with the `Quaternion.from_axis_angle` class method.
We also will create the `idx_dimension` variable which shrinks subsequent matrices and vectors to use only the first two ($x$ and $y$) components.

In [ ]:
quat = lambda q: Quaternion(*q) if isinstance(q, np.ndarray) else Quaternion.from_axis_angle([0, 0, 1], q)
idx_dimension = lambda orientation: 3 if isinstance(orientation, np.ndarray) else 2

In [ ]:
quaternion_2d = quat(orientation_2d)
display(quaternion_2d)

In [ ]:
quaternion_3d = quat(orientation_3d)
display(quaternion_3d)

## Equation for gaussian

The Gaussian equation that is created below uses the variance term _not-squared_, while in the plot I square this term.
There is reason for this; the variance that I will use below is going to be of the form

$\texttt{variance} = \sum_{i=1}^3 \left( \frac{x_i - \bar{x}_i}{s_i} \right)^2$

where $x_i$ is a position coordinate for where we are measuring, $\bar{x}_i$ is the position coordinate for the center of the Gaussian, and $s_i$ is a scale coefficient representing one standard deviation in the direction of that coordinate.
Traditionally, we would evaluate the Gaussian at some count of standard deviations, which would derive from the square-root of the above expression.
To avoid taking a square-root, just to subsequently square the term again, we create the Gaussian as a function of the variance directly.
However, the plot below shows an evaluation of the Gaussian with respect to standard deviations.

In [ ]:
gaussian = lambda variance: exp(-0.5 * variance)
stdev = np.linspace(-3, 3, 121);
plt.plot(stdev, np.exp(-0.5 * stdev**2));
plt.xlabel("Standard Deviations");
plt.ylabel("Gaussian");

Here we create a function that obtains the rotation matrix from the quaternion, along with some simplification steps.
First, we substitute a value of 1 for the quaternion norm, as we will ensure that it is always a proper unit quaternion.
Second, we call the standard `simplify()` function.

In [ ]:
def rotation_matrix_eqn(orientation):
    quaternion = quat(orientation)
    rotation_matrix = quaternion.to_rotation_matrix().subs(quaternion.norm(), 1)
    rotation_matrix.simplify()
    idx = idx_dimension(orientation)
    return rotation_matrix[:idx, :idx]

In [ ]:
rotation_eqn_2d = rotation_matrix_eqn(orientation_2d)
rotation_fcn_2d = lambdify(θ, rotation_eqn_2d)
display(rotation_eqn_2d)
display(rotation_fcn_2d(0.5))

In [ ]:
rotation_eqn_3d = rotation_matrix_eqn(orientation_3d)
rotation_fcn_3d = lambdify(orientation_3d, rotation_eqn_3d)
display(rotation_eqn_3d)
display(rotation_fcn_3d(0.707, 0.707, 0, 0))

In [ ]:
def variance_eqn(orientation):
    rotation_matrix = rotation_matrix_eqn(orientation)
    idx = idx_dimension(orientation)
    dx = rotation_matrix[:idx, :idx] @ (eval_at_position[:idx] - position[:idx])
    s = dx / scale[:idx]
    return np.sum(s**2)

In [ ]:
variance_eqn_2d = variance_eqn(orientation_2d)
variance_fcn_2d = lambdify(splat_properties_2d + eval_coords_2d, variance_eqn_2d)
display(variance_eqn_2d)
display(variance_fcn_2d(*splat_properties_2d_example))

In [ ]:
variance_eqn_3d = variance_eqn(orientation_3d)
variance_fcn_3d = lambdify(splat_properties_3d + eval_coords_3d, variance_eqn_3d)
display(variance_eqn_3d)
display(variance_fcn_3d(*splat_properties_3d_example))

Velocity away from the center of the Gaussian takes the form

$\boldsymbol{x} = (x, y, z, u, v, w, s_x, s_y, s_z, q_w, q_x, q_y, q_z)$

$\boldsymbol{x}_p = (x_p, y_p, z_p)$

$\boldsymbol{v(\boldsymbol{x}, \boldsymbol{x}_p)} = \boldsymbol{v_0(\boldsymbol{x})} g(\boldsymbol{x}, \boldsymbol{x}_p)$

In [ ]:
def velocity_eqn(orientation):
    idx = idx_dimension(orientation)
    variance = variance_eqn(orientation)
    return velocity[:idx] * gaussian(variance)

In [ ]:
velocity_eqn_2d = velocity_eqn(orientation_2d)
velocity_fcn_2d = lambdify(splat_properties_2d + eval_coords_2d, list(velocity_eqn_2d))
display(Matrix(velocity_eqn_2d))
display(velocity_fcn_2d(*splat_properties_2d_example))

In [ ]:
velocity_eqn_3d = velocity_eqn(orientation_3d)
velocity_fcn_3d = lambdify(splat_properties_3d + eval_coords_3d, list(velocity_eqn_3d))
#display(Matrix(velocity_eqn_3d))
display(velocity_fcn_3d(*splat_properties_3d_example))

In [ ]:
diff_variance_eqn_2d = [diff(variance_eqn_2d, variable) for variable in splat_properties_2d]
diff_variance_fcn_2d = lambdify(splat_properties_2d + eval_coords_2d, list(diff_variance_eqn_2d))
#display(Matrix(diff_variance_eqn_2d))
display(diff_variance_fcn_2d(*splat_properties_2d_example))

In [ ]:
diff_variance_eqn_3d = [diff(variance_eqn_3d, variable) for variable in splat_properties_3d]
diff_variance_fcn_3d = lambdify(splat_properties_3d + eval_coords_3d, list(diff_variance_eqn_3d))
#display(Matrix(diff_variance_eqn_3d))
display(diff_variance_fcn_3d(*splat_properties_3d_example))

In [ ]:
#diff_gaussian_eqn_2d = -0.5 * diff_variance_eqn_2d * gaussian(variance_eqn_2d)
#display(diff_gaussian_eqn_2d)

### Velocity derivative equations

Velocity is 1x2 or 1x3

Variance derivates are 1xN

$
\frac{\partial \boldsymbol{v(\boldsymbol{x}, \boldsymbol{x}_p)} }{\partial x_i}
= 
\frac{\partial \boldsymbol{v_0(\boldsymbol{x})}}{\partial x_i}
g(\boldsymbol{x}, \boldsymbol{x}_p)
+
\boldsymbol{v_0(\boldsymbol{x})}
\frac{\partial g(\boldsymbol{x}, \boldsymbol{x}_p)}{\partial x_i}
$

In [ ]:
def velocity_derivatives(ndim):
    if ndim == 2:
        return Matrix(np.concatenate([np.zeros([2, 2]), np.diag([1, 1]), np.zeros([3, 2])]))
    elif ndim == 3:
        return Matrix(np.concatenate([np.zeros([3, 3]), np.diag([1, 1, 1]), np.zeros([7, 3])]))
    
# TODO: add the second part
display(velocity_derivatives(2)*gaussian(variance_eqn_2d))

## Create Splatting Class

In [ ]:
class AeroSplat:
    position = np.zeros(2)
    velocity = np.zeros(2)
    scale = np.ones(2)
    orientation = 0 # np.array([0, 0, 0, 1])
    
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def __repr__(self):
        return f"AeroSplat(position={self.position}, velocity={self.velocity}, scale={self.scale}, orientation={self.orientation})"
    
    @property
    def is_symbolic(self):
        # TODO: detect if the states are symbolic or numpy/numeric
        return
    
    @property
    def quaternion(self):
        return quat(self.orientation)
    
    @property
    def rotation_matrix(self):
        R = self.quaternion.to_rotation_matrix()
        R.simplify()
        return R[:2, :2] if len(self.position) == 2 else R
    
    @property
    def scale_matrix(self):
        return np.diag(self.scale)
    
    def velocity_at(self, position):
        dx = self.rotation_matrix @ (position - self.position)
        s = dx / self.scale
        variance = np.sum(s**2)
        try:
            return self.velocity * np.exp(-0.5 * variance)
        except:
            return self.velocity * exp(-0.5 * variance)
    
    def differential_velocity_at(self, position):
        return np.zeros(3)
        

In [ ]:
a = AeroSplat(
    position=np.array([x, y, z]), 
    velocity=np.array([u, v, w]),
    scale=np.array([sx, sy, sz]),
    orientation=np.array([qx, qy, qz, qw])
)
display(a)
display(a.quaternion)
display(a.rotation_matrix)
display(Matrix(a.velocity_at(np.array([xp, yp, zp]))))
#display(Matrix(a.velocity_at(np.ones(3))))

In [ ]:
class LineBoundary:
    point0 = np.zeros(3)
    point1 = np.zeros(3)
    
    def __init__(self, point0=np.zeros(3), point1=np.zeros(3)):
        self.point0 = point0
        self.point1 = point1
        
    def __repr__(self):
        return f"LineBoundary(point0={self.point0}, point1={self.point1})"

In [ ]:
line = LineBoundary(point1=np.ones(3))

print(line)